# Julia 機器學習：GLM 線性迴歸

## 作業 027：波士頓房價預測資料集

請使用 GLM 中的模型，建立一個預測模型來預測波士頓的房價。

In [3]:
using GLM, RDatasets, MLDataUtils

## 讀取資料

In [4]:
boston = dataset("MASS", "Boston")
first(boston, 10)

,Crim,Zn,Indus,Chas,NOx,Rm,Age,Dis,Rad,Tax
,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Float64,Int64,Int64
1,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.09,1,296
2,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242
3,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242
4,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222
5,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222
6,0.02985,0.0,2.18,0,0.458,6.43,58.7,6.0622,3,222
7,0.08829,12.5,7.87,0,0.524,6.012,66.6,5.5605,5,311
8,0.14455,12.5,7.87,0,0.524,6.172,96.1,5.9505,5,311
9,0.21124,12.5,7.87,0,0.524,5.631,100.0,6.0821,5,311


## 切分訓練資料及測試資料

In [5]:
indecies = MLDataUtils.shuffleobs(collect(1:nrow(boston)))
train_ind, test_ind = MLDataUtils.splitobs(indecies, at = 0.8);

In [6]:
train = boston[train_ind, :]
test = boston[test_ind, :]

,Crim,Zn,Indus,Chas,NOx,Rm,Age,Dis,Rad,Tax
,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Float64,Int64,Int64
1,0.05083,0.0,5.19,0,0.515,6.316,38.1,6.4584,5,224
2,10.6718,0.0,18.1,0,0.74,6.459,94.8,1.9879,24,666
3,5.66998,0.0,18.1,1,0.631,6.683,96.8,1.3567,24,666
4,0.62356,0.0,6.2,1,0.507,6.879,77.7,3.2721,8,307
5,7.40389,0.0,18.1,0,0.597,5.617,97.9,1.4547,24,666
6,0.01096,55.0,2.25,0,0.389,6.453,31.9,7.3073,1,300
7,4.03841,0.0,18.1,0,0.532,6.229,90.7,3.0993,24,666
8,0.05059,0.0,4.49,0,0.449,6.389,48.0,4.7794,3,247
9,1.13081,0.0,8.14,0,0.538,5.713,94.1,4.233,4,307


In [8]:
describe(boston)

,variable,mean,min,median,max,nunique,nmissing,eltype
,Symbol,Float64,Real,Float64,Real,Nothing,Nothing,DataType
1,Crim,3.61352,0.00632,0.25651,88.9762,,,Float64
2,Zn,11.3636,0.0,0.0,100.0,,,Float64
3,Indus,11.1368,0.46,9.69,27.74,,,Float64
4,Chas,0.06917,0,0.0,1,,,Int64
5,NOx,0.554695,0.385,0.538,0.871,,,Float64
6,Rm,6.28463,3.561,6.2085,8.78,,,Float64
7,Age,68.5749,2.9,77.5,100.0,,,Float64
8,Dis,3.79504,1.1296,3.20745,12.1265,,,Float64
9,Rad,9.54941,1,5.0,24,,,Int64


## 線性迴歸模型 

In [7]:
ols = GLM.lm(@formula(MedV ~ Crim + Zn + Indus + Chas + NOx + Rm + Age + Dis + Rad + Tax + PTRatio + Black + LStat), train)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Array{Float64,1}},GLM.DensePredChol{Float64,LinearAlgebra.Cholesky{Float64,Array{Float64,2}}}},Array{Float64,2}}

MedV ~ 1 + Crim + Zn + Indus + Chas + NOx + Rm + Age + Dis + Rad + Tax + PTRatio + Black + LStat

Coefficients:
───────────────────────────────────────────────────────────────────────────────────────
                 Estimate  Std. Error     t value  Pr(>|t|)      Lower 95%    Upper 95%
───────────────────────────────────────────────────────────────────────────────────────
(Intercept)   37.4526      5.73537      6.53011      <1e-9    26.1766       48.7286    
Crim          -0.103698    0.0348719   -2.97369      0.0031   -0.172258     -0.0351384 
Zn             0.0399437   0.0153273    2.60604      0.0095    0.00980939    0.070078  
Indus          0.015991    0.0682489    0.234304     0.8149   -0.11819       0.150172  
Chas           2.69872     1.07505      2.51031      0.0125    0.585106      4.81233   
NOx          -

## 預測 

In [9]:
predict(ols, test)

101-element Array{Union{Missing, Float64},1}:
 22.570911332614482 
 12.67315400034904  
 32.52983326331596  
 32.37599122614572  
 13.244295282638046 
 27.886246358650148 
 22.677997952322713 
 25.188574093820478 
 11.480971453537604 
 20.711161118445816 
 40.438320076065864 
 31.08149008262544  
 22.300208054001065 
  ⋮                 
 26.19807449323222  
 17.77583788243568  
 19.54677038019056  
 20.68072217556943  
 35.975482296133755 
 27.497411464198464 
 16.881278298736852 
 13.650867498335602 
 27.9663962876047   
 18.49766472668272  
  2.6784423836651996
 23.938417151552443 

## 模型評估 

In [10]:
GLM.r²(ols)

0.7445712968144829

In [11]:
GLM.adjr²(ols)

0.73607878238632